# From symbolic potentials to lambdified gradients

1. This notebook examines the CSE feature of sympy - common subexpression elimination to
avoid repeated evaluation of of equal expressions. 

2. The array based evaluation that can be used for vectorized execution of lambdified expressions is examined as well.

In [ ]:
%matplotlib widget 
import sympy as sp # CAS used for symbolic derivations 
sp.init_printing()
import matplotlib.pyplot as plt # plotting package
from matplotlib import cm # color maps for plotting
from sympy.utilities.codegen import codegen # code generation package
import numpy as np # array based numerical package 
np.seterr(divide='ignore', invalid='ignore') # suppress warnings on division by zero
from bmcs_utils.api import Cymbol
from sympy import Heaviside
import copy

In [ ]:
H = Heaviside
def get_dirac_delta(x):
    return 0
numpy_dirac =[{'DiracDelta': get_dirac_delta }, 'numpy']

In [ ]:
E_T = Cymbol(r'E_{\mathrm{T}}', codename='E_T_', real=True, nonnegative=True)
gamma_T = Cymbol(r'\gamma_{\mathrm{T}}', codename='gamma_T_', real=True)
K_T = Cymbol(r'K_{\mathrm{T}}', codename='K_T_', real=True)
S_T = Cymbol(r'S_{\mathrm{T}}', codename='S_T_', real=True, nonnegative=True)
r_T = Cymbol(r'r_{\mathrm{T}}', codename='r_T_', real=True, nonnegative=True)
c_T = Cymbol(r'c_{\mathrm{T}}', codename='c_T_', real=True, nonnegative=True)
eta_T = Cymbol(r'\eta_{\mathrm{T}}', codename='eta_T_', real=True, nonnegative=True)

E_N = Cymbol(r'E_{\mathrm{N}}', codename='E_N_', real=True, nonnegative=True)
gamma_N = Cymbol(r'\gamma_{\mathrm{N}}', codename='gamma_N_', real=True)
K_N = Cymbol(r'K_{\mathrm{N}}', codename='K_N_', real=True)
S_N = Cymbol(r'S_{\mathrm{N}}', codename='S_N_', real=True, nonnegative=True)
r_N = Cymbol(r'r_{\mathrm{N}}', codename='r_N_', real=True, nonnegative=True)
c_N = Cymbol(r'c_{\mathrm{N}}', codename='c_N_', real=True, nonnegative=True)

eta_N = Cymbol(r'\eta_{\mathrm{N}}', codename='eta_N_', real=True, nonnegative=True)
zeta = Cymbol('zeta', codename='zeta_', real=True, nonnegative=True)

d_N = Cymbol(r'd_{\mathrm{N}}', codename='d_N_', real=True, nonnegative=True)
alpha_therm = Cymbol(r'\alpha_{\vartheta}', codename='alpha_therm_', real=True, nonnegative=True)
# temperature 
C_v = Cymbol(r'C_{\mathrm{v}}', codename='C_v_', real=True, nonnegative=True)
T_0 = Cymbol(r'\vartheta_0', codename='T_0_', real=True, nonnegative=True)
beta = Cymbol(r'\beta', codename='beta_', real=True, nonnegative=True)

## Parameters of the threshold function

In [ ]:
f_t = Cymbol(r'f_\mathrm{Nt}', codename='f_t_')
f_c = Cymbol(r'f_\mathrm{Nc}', codename='f_c_')
f_c0 = Cymbol(r'f_\mathrm{Nc0}', codename='f_c0_')
f_s = Cymbol(r'f_\mathrm{T}', codename='f_s_')
m = Cymbol(r'm', codename='m_', real=True, nonnegative=True)

In [ ]:
sp_vars = (E_T, gamma_T, K_T, S_T, c_T, f_s, E_N, gamma_N, K_N, S_N, c_N, m, f_t, f_c, f_c0, 
           r_N, r_T, eta_N, eta_T, zeta, C_v, T_0, d_N, alpha_therm, beta)
sp_vars

In [ ]:
T = Cymbol(r'\vartheta', codename='T_', real=True)
Gamma = sp.exp(-beta * (T - T_0))
Gamma

In [ ]:
E_ab = sp.Matrix([[E_N, 0, 0],
                  [0, E_T, 0],
                  [0, 0, E_T]])

In [ ]:
u_N = Cymbol(r'u_\mathrm{N}', codename='u_N_', real=True)
u_Tx = Cymbol(r'u_\mathrm{Tx}', codename='u_Tx_', real=True)
u_Ty = Cymbol(r'u_\mathrm{Ty}', codename='u_Ty_', real=True)
u_Ta = sp.Matrix([u_Tx, u_Ty])
u_a = sp.Matrix([u_N, u_Tx, u_Ty])
sig_N = Cymbol(r'\sigma_\mathrm{N}', codename='sig_N_', real=True)
sig_Tx = Cymbol(r'\sigma_\mathrm{Tx}', codename='sig_Tx_', real=True)
sig_Ty = Cymbol(r'\sigma_\mathrm{Ty}', codename='sig_Ty_', real=True)
sig_Ta = sp.Matrix([sig_Tx, sig_Ty])
sig_a = sp.Matrix([sig_N, sig_Tx, sig_Ty])

## Internal variables

In [ ]:
u_p_N = Cymbol(r'u_\mathrm{N}^\mathrm{p}', codename='u_p_N_', real=True)
u_p_Tx = Cymbol(r'u_\mathrm{Tx}^\mathrm{p}', codename='u_p_Tx_', real=True)
u_p_Ty = Cymbol(r'u_\mathrm{Ty}^\mathrm{p}', codename='u_p_Ty_', real=True)
u_p_Ta = sp.Matrix([u_p_Tx, u_p_Ty])
u_p_a = sp.Matrix([u_p_N, u_p_Tx, u_p_Ty])
sig_p_N = Cymbol(r'\sigma^\mathrm{p}_\mathrm{N}', codename='sig_p_N_', real=True)
sig_p_Tx = Cymbol(r'\sigma^\mathrm{p}_\mathrm{Tx}', codename='sig_p_Tx_', real=True)
sig_p_Ty = Cymbol(r'\sigma^\mathrm{p}_\mathrm{Ty}', codename='sig_p_Ty_', real=True)
sig_p_Ta = sp.Matrix([sig_p_Tx, sig_p_Ty])
sig_p_a = sp.Matrix([sig_p_N, sig_p_Tx, sig_p_Ty])

In [ ]:
omega_N = Cymbol(r'\omega_\mathrm{N}', codename='omega_N_', real=True)
omega_T = Cymbol(r'\omega_\mathrm{T}', codename='omega_T_', real=True)
omega_ab = sp.Matrix([[omega_N, 0, 0],
                      [0, omega_T, 0],
                      [0, 0, omega_T]])
omega_a = sp.Matrix([omega_N, omega_T])
Y_N = Cymbol(r'Y_\mathrm{N}', codename='Y_N_', real=True)
Y_T = Cymbol(r'Y_\mathrm{T}', codename='Y_T_', real=True)
Y_a = sp.Matrix([Y_N, Y_T])

In [ ]:
z_N = Cymbol(r'z_\mathrm{N}', codename='z_N_', real=True)
z_T = Cymbol(r'z_\mathrm{T}', codename='z_T_', real=True)
K_ab = sp.Matrix([[K_N, 0],
                  [0, K_T]])
z_a = sp.Matrix([z_N, z_T])
Z_N = Cymbol(r'Z_\mathrm{N}', codename='Z_N_', real=True)
Z_T = Cymbol(r'Z_\mathrm{T}', codename='Z_T_', real=True)
Z_a = sp.Matrix([Z_N, Z_T])

In [ ]:
alpha_N = Cymbol(r'\alpha_\mathrm{N}', codename='alpha_N_', real=True, nonnegative=True)
alpha_Tx = Cymbol(r'\alpha_\mathrm{Tx}', codename='alpha_Tx_', real=True, nonnegative=True)
alpha_Ty = Cymbol(r'\alpha_\mathrm{Ty}', codename='alpha_Ty_', real=True, nonnegative=True)
gamma_ab = sp.Matrix([[gamma_N, 0, 0],
                      [0, gamma_T, 0],
                      [0, 0, gamma_T]])
alpha_Ta = sp.Matrix([alpha_Tx, alpha_Ty])
alpha_a = sp.Matrix([alpha_N, alpha_Tx, alpha_Ty])
X_N = Cymbol(r'X_\mathrm{N}', codename='X_N_', real=True, nonnegative=True)
X_Tx = Cymbol(r'X_\mathrm{Tx}', codename='X_Ty_', real=True, nonnegative=True)
X_Ty = Cymbol(r'X_\mathrm{Ty}', codename='X_Tx_', real=True, nonnegative=True)
X_Ta = sp.Matrix([X_Tx, X_Ty])
X_a = sp.Matrix([X_N, X_Tx, X_Ty])

In [ ]:
Eps_vars = u_p_a, omega_a, z_a, alpha_a
Sig_vars = sig_p_a, Y_a, Z_a, X_a
Eps_vars, Sig_vars

In [ ]:
Eps = sp.BlockMatrix([Eps_i.T for Eps_i in Eps_vars]).T
Sig = sp.BlockMatrix([Sig_i.T for Sig_i in Sig_vars]).T
Eps, Sig

In [ ]:
# Eps = [Eps_i.T for Eps_i in Eps_vars]
# Sig = [Sig_i.T for Sig_i in Sig_vars]
# Eps, Sig

In [ ]:
u_el_a = u_a - u_p_a
E_eff_ab = (sp.eye(3) - omega_ab) * E_ab
E_eff_ab

In [ ]:
u_el_a

In [ ]:
U_T_ = ( (1 - omega_N) * E_N * alpha_therm * (T - T_0) * (u_N - u_p_N) * d_N )
U_e_ = sp.Rational(1,2) * (u_el_a.T * E_eff_ab * u_el_a)[0]
U_p_ = sp.Rational(1,2) * (z_a.T * K_ab * z_a + alpha_a.T * gamma_ab * alpha_a)[0]
TS_ = C_v * (T - T_0) **2 / (2 * T_0)
F_ = U_e_ + U_p_ + U_T_ - TS_
F_

In [ ]:
dF_dEps_ = sp.BlockMatrix([sp.diff(F_, var).T for var in Eps.blocks]).T
dF_dEps_

In [ ]:
Sig_signs = [-1, -1, 1, 1]
Sig_ = sp.BlockMatrix([(sign_i_ * dF_dEps_i_).T for sign_i_, dF_dEps_i_ in zip(Sig_signs, dF_dEps_.blocks)]).T
Sig_

In [ ]:
get_Sig_ = sp.lambdify((u_a, T, Eps.blocks, Sig.blocks)+sp_vars+('**kw',), list(Sig_.blocks), numpy_dirac, 
                        cse=True)

In [ ]:
Diss_a = sp.BlockMatrix([sp.Matrix(T*dF_dEps_i.diff(T) - dF_dEps_i).T for dF_dEps_i in dF_dEps_.blocks]).T
Diss_a

In [ ]:
Eps

In [ ]:
dSig_dEps_ = sp.BlockMatrix([[sp.Matrix(Sig_i.diff(Eps_i)[:,0,:,0]) for Sig_i in Sig_.blocks] for Eps_i in Eps.blocks])
dSig_dEps_

## Threshold function

### Double cap domain

In [ ]:
%%capture
%run threshold_function_f_df.ipynb

In [ ]:
sig_eff_Tx = sp.Function(r'\sigma^{\mathrm{eff}}_{\mathrm{T}x}')(sig_p_Tx, omega_T)
sig_eff_Ty = sp.Function(r'\sigma^{\mathrm{eff}}_{\mathrm{T}y}')(sig_p_Ty, omega_T)
sig_eff_N = sp.Function(r'\sigma^{\mathrm{eff}}_{\mathrm{N}}')(sig_p_N, omega_N)
q_Tx = sp.Function(r'q_Tx')(sig_eff_Tx,X_Tx)
q_Ty = sp.Function(r'q_Ty')(sig_eff_Ty,X_Ty)
q_N = sp.Function(r'q_N')(sig_eff_N)
norm_q_T = sp.sqrt(q_Tx*q_Tx + q_Ty*q_Ty)
subs_q_T = {q_Tx: (sig_eff_Tx - X_Tx), q_Ty: (sig_eff_Ty - X_Ty)}
subs_q_N = {q_N: sig_eff_N}
subs_sig_eff = {sig_eff_Tx: sig_p_Tx / (1-omega_T),
                  sig_eff_Ty: sig_p_Ty / (1-omega_T),
                  sig_eff_N: sig_p_N / (1-omega_N)
                 }
f_ = (f_solved_
      .subs({x: q_N, y: norm_q_T})
      .subs(subs_q_T)
      .subs(subs_q_N)
      .subs(subs_sig_eff)
      .subs(f_s,((f_s+Z_T) * Gamma))
      .subs(f_t,f_t)
      .subs(f_c0,f_c0)
      .subs(f_c,f_c)
      )
f_

In [ ]:
get_f_ = sp.lambdify((T, Eps.blocks, Sig.blocks)+sp_vars+('**kw',), f_, numpy_dirac, 
                        cse=True)


In [ ]:
material_params = dict(
    E_T_=1, gamma_T_=1, K_T_=1, S_T_=1, c_T_=1, f_s_=1, 
    E_N_=1, gamma_N_=1, K_N_=1, S_N_=0.5, c_N_=1, m_=0.1, f_t_=1, f_c_=20, f_c0_=10, 
    r_N_=2, r_T_=2, zeta_=1., eta_N_=1, eta_T_=1, d_N_=0, alpha_therm_=1e-5, T_0_=20, C_v_=1, beta_=1
)

### Single value check

In [ ]:
_u_a = [3, 0, 0]
_Eps_0_B_00 = [np.zeros(Eps_i.shape[:-1], np.float_) for Eps_i in Eps.blocks]
_Sig_0_B_00 = get_Sig_(_u_a, 20, _Eps_0_B_00, _Eps_0_B_00, **material_params)
_Sig_0_B_00 = [_Sig_0_i_00[:, 0] for _Sig_0_i_00 in _Sig_0_B_00]
_f_blocks_00 = get_f_(20, _Eps_0_B_00, _Sig_0_B_00, **material_params)
_f_blocks_00

### Countours in the apparent stress domain

In [ ]:
sig_p_range, tau_p_range = np.mgrid[-22:3:500j, -5:5:500j]
_Eps_0_B_IJ = [np.zeros(Eps_i.shape[:-1] + sig_p_range.shape, np.float_) for Eps_i in Eps.blocks]
_Sig_0_B_IJ = [np.zeros(Sig_i.shape[:-1] + sig_p_range.shape, np.float_) for Sig_i in Sig.blocks]
_Sig_0_B_IJ[0][0] = sig_p_range
_Sig_0_B_IJ[0][2] = tau_p_range
_f_blocks_IJ = get_f_(20, _Eps_0_B_IJ, _Sig_0_B_IJ, **material_params)
fig = plt.figure(figsize=(8, 5), tight_layout=True)
ax = fig.add_subplot(1, 1, 1) # , projection='3d')
ax.contour(sig_p_range, tau_p_range, _f_blocks_IJ, levels=[0]);
ax.axis('equal');

### Countours in control displacement/strain domain 

In [ ]:
u_N_range, u_T_range = np.mgrid[-22:3:500j, -5:5:500j]
_u_IJ = np.zeros((3,) + u_N_range.shape, np.float_)
_u_IJ[0] = u_N_range
_u_IJ[2] = u_T_range
_Eps_0_B_IJ = [np.zeros(Eps_i.shape[:-1]  + u_N_range.shape, np.float_) for Eps_i in Eps.blocks]
_Sig_0_B_IJ = get_Sig_(_u_IJ, 20, _Eps_0_B_IJ, _Eps_0_B_IJ, **material_params)
_Sig_0_B_IJ = [_Sig_0_i_00[:, 0] for _Sig_0_i_00 in _Sig_0_B_IJ]
_f_IJ = get_f_(20, _Eps_0_B_IJ, _Sig_0_B_IJ, **material_params)
fig, ax = plt.subplots(1, 1, figsize=(8, 5), tight_layout=True)
ax.contour(u_N_range, u_T_range, _f_IJ, levels=[0]);
ax.axis('equal');

In [ ]:
df_dSig_list = [f_.diff(Sig_i) for Sig_i in Sig.blocks]
df_dEps_list = [f_.diff(Eps_i) for Eps_i in Eps.blocks]
df_dSig_ = sp.BlockMatrix([[df_dSig_i] for df_dSig_i in df_dSig_list])
df_dEps_ = sp.BlockMatrix([[df_dEps_i] for df_dEps_i in df_dEps_list])

### Circular elastic domain

In [ ]:
# sig_eff_a = sig_p_a / (1 - omega_T)
# q = sig_eff_a - X_a
# norm_q = sp.sqrt((q.T * q)[0])
# f_ = norm_q - f_N_t
# df_dSig_list = [f_.diff(Sig_i) for Sig_i in Sig]
# df_dEps_list = [f_.diff(Eps_i) for Eps_i in Eps]

# df_dSig_ = sp.BlockMatrix([[df_dSig_i] for df_dSig_i in df_dSig_list])
# df_dEps_ = sp.BlockMatrix([[df_dEps_i] for df_dEps_i in df_dEps_list])

## Dissipation potential

In [ ]:
S_NT = sp.sqrt(S_N*S_T)
c_NT = sp.sqrt(c_N*c_T)
r_NT = sp.sqrt(r_N*r_T)
omega_NT = 1 - sp.sqrt((1-omega_N)*(1-omega_T))
phi_N = (1 - omega_N)**c_N * S_N / (r_N+1) * (Y_N / S_N)**(r_N+1)
phi_T = (1 - omega_T)**c_T * S_T / (r_T+1) * (Y_T / S_T)**(r_T+1)
phi_NT = (1 - omega_NT)**c_NT * S_NT / (r_NT+1) * ((Y_N + Y_T)/(S_NT))**(r_NT+1)
phi_ext_ = ((1 - zeta)*(phi_N + phi_T) + zeta*phi_NT)
phi_ = f_ + phi_ext_ 
Phi_list = [-sign_i_ * phi_.diff(Sig_i_) for sign_i_, Sig_i_ in zip(Sig_signs, Sig.blocks)]
Phi_ = sp.BlockMatrix([[Phi_i] for Phi_i in Phi_list])
phi_ext_.diff(Sig.as_explicit())

## Time dependency

In [ ]:
t_relax_N_ = eta_N / (E_N + gamma_N)
t_relax_T_ = eta_T / (E_T + K_T + gamma_T)
t_relax_ = sp.Matrix([
                    t_relax_T_,
                    t_relax_T_,
                    t_relax_N_,
                    t_relax_T_,
                    t_relax_T_,
                    t_relax_T_,
                    ])

t_relax_

## Return mapping - dissipation parameter

### Predictor
$$\left.
\frac{\partial f}{\partial{\lambda}}  
\right|_k 
=
\left.
\frac{\partial f}{\partial{\boldsymbol{ \mathcal{E}}}}  
\right|_k 
\left.
\frac{\partial {\boldsymbol{ \mathcal{E}}}}{\partial \lambda}
\right|_k =
\left.
\frac{\partial f}{\partial{\boldsymbol{ \mathcal{E}}}}  
\right|_k 
\boldsymbol{\Phi}_k \\
\left.
\frac{\partial f}{\partial{\boldsymbol{ \mathcal{E}}}}  
\right|_k 
=
\left. \frac{\partial f}{ \partial \boldsymbol{\mathcal{S}}}\right|_{k}
\left. \frac{\partial \boldsymbol{\mathcal{S}}}{\partial \boldsymbol{\mathcal{E}}}\right|_{k}
+
\left. \frac{\partial^{\mathrm{dir}} f}{ \partial^{\mathrm{dir}} \boldsymbol{\mathcal{E}}}\right|_{k}
$$

In [ ]:
df_dSig_.shape, dSig_dEps_.shape, df_dEps_.shape, Phi_.shape

### Version 0 - without cse

In [ ]:
df_dlambda_ = ((dSig_dEps_.as_explicit() * df_dSig_.as_explicit() + df_dEps_.as_explicit()).T * Phi_.as_explicit())[0, 0]
get_df_dlambda_0_ = sp.lambdify((u_a, T, Eps.blocks, Sig.blocks) + sp_vars + ('**kw',), df_dlambda_, 
                              numpy_dirac, cse=True)

In [ ]:
# import inspect
# source_code = inspect.getsource(get_df_dlambda_0_)
# print(source_code)

In [ ]:
_u_a = [1e-6, 1e-6, 0]
_Sig_0_B_00 = get_Sig_(_u_a, 20, _Eps_0_B_00, _Eps_0_B_00, **material_params)
_Sig_0_B_00 = [_Sig_0_i_00[:, 0] for _Sig_0_i_00 in _Sig_0_B_00]
_f = get_f_(20, _Eps_0_B_00, _Sig_0_B_00, **material_params)
_df_dlambda = get_df_dlambda_0_(_u_a, 20, _Eps_0_B_00, _Sig_0_B_00, **material_params)
_f, _df_dlambda

In [ ]:
u_N_range, u_T_range = np.mgrid[-22:3:500j, -5:5:500j]
_u_IJ = np.zeros((3,) + u_N_range.shape, np.float_)
_u_IJ[0] = u_N_range
_u_IJ[2] = u_T_range
_Eps_0_B_IJ = [np.zeros(Eps_i.shape[:-1]  + u_N_range.shape, np.float_) for Eps_i in Eps.blocks]
_Sig_0_B_IJ = get_Sig_(_u_IJ, 20, _Eps_0_B_IJ, _Eps_0_B_IJ, **material_params)
_Sig_0_B_IJ = [_Sig_0_i_00[:, 0] for _Sig_0_i_00 in _Sig_0_B_IJ]
_f_IJ = get_f_(20, _Eps_0_B_IJ, _Sig_0_B_IJ, **material_params)
_df_dlambda_IJ = get_df_dlambda_0_(_u_IJ, 20, _Eps_0_B_IJ, _Sig_0_B_IJ, **material_params)
fig, ax = plt.subplots(1, 1, figsize=(7, 3), tight_layout=True)
fig.canvas.header_visible=False
ax.contour(sig_p_range, tau_p_range, _f_IJ, levels=[0]);
ax.contourf(sig_p_range, tau_p_range, _df_dlambda_IJ);
ax.axis('equal');

### Version 1 - cse with chain rule

In [ ]:
def lambdify_cs(vars, sp_vars, cs_list):
    """Generate the function that will take all the input parameters and return
    the sequence of all interim variables
    """
    cs_vars, cs_exprs = zip(*cs_list)

    get_x_i = [sp.lambdify(vars + cs_vars[:i] + sp_vars + ('**kw',), cs_expr)
                           for i, cs_expr in enumerate(cs_exprs)]
    def get_interim_vars(*params, **kw):
        parameters = list(params)
        results = []
        for cs_var, f in zip(cs_vars, get_x_i):
            result = f(*parameters, **kw)
            results.append(result)
            parameters.append(result)

        return results
    return tuple(cs_vars), get_interim_vars

In [ ]:
df_dSig_dEps_reduced_, (dSig_dEps_cse, df_dSig_cse, df_dEps_cse, Phi_cse) = sp.cse(
    [expr.as_explicit() for expr in [dSig_dEps_, df_dSig_, df_dEps_, Phi_]])
cs_vars, get_lambda_interim_cs = lambdify_cs((u_a, T, Eps.blocks, Sig.blocks), sp_vars, df_dSig_dEps_reduced_)
df_dlambda_ = ((dSig_dEps_cse * df_dSig_cse + df_dEps_cse).T * Phi_cse)[0, 0]
get_df_dlambda_1_ = sp.lambdify((Eps.blocks, Sig.blocks) + cs_vars + sp_vars + ('**kw',), df_dlambda_, numpy_dirac)

In [ ]:
len(cs_vars), df_dlambda_

In [ ]:
_u_a = [1e-6, 1e-6, 0]
_Sig_0_B_00 = get_Sig_(_u_a, 20, _Eps_0_B_00, _Eps_0_B_00, **material_params)
_Sig_0_B_00 = [_Sig_0_i_00[:, 0] for _Sig_0_i_00 in _Sig_0_B_00]
_f = get_f_(20, _Eps_0_B_00, _Sig_0_B_00, **material_params)
_x_i_cse = get_lambda_interim_cs(_u_a, 20, _Eps_0_B_00, _Sig_0_B_00, **material_params)
_df_dlambda = get_df_dlambda_1_(_Eps_0_B_00, _Sig_0_B_00, *_x_i_cse, **material_params)
_f, _df_dlambda

In [ ]:
_f_IJ = get_f_(20, _Eps_0_B_IJ, _Sig_0_B_IJ, **material_params)
_x_i_cse_IJ = get_lambda_interim_cs(_u_IJ, 20, _Eps_0_B_IJ, _Sig_0_B_IJ, **material_params)
_df_dlambda_IJ = get_df_dlambda_1_(_Eps_0_B_IJ, _Sig_0_B_IJ, *_x_i_cse_IJ, **material_params)
fig, ax = plt.subplots(1, 1, figsize=(7, 3), tight_layout=True)
fig.canvas.header_visible=False
ax.contour(sig_p_range, tau_p_range, _f_IJ, levels=[0]);
ax.contourf(sig_p_range, tau_p_range, _df_dlambda_IJ);
ax.axis('equal');

### Version 2 substitute first and differentiate w.r.t. $\mathcal{S}$ and cse

In [ ]:
subs_Sig_Eps_ = dict(zip(Sig.as_explicit(), Sig_.as_explicit()))
f_Sig_Eps_ = f_.subs(subs_Sig_Eps_)
df_dSig_dEps_ = f_Sig_Eps_.diff(Eps.as_explicit())
df_dSig_dEps_reduced_, (f_cse, df_dSig_dEps_cse, Phi_cse) = sp.cse(
    [f_, df_dSig_dEps_.as_explicit(), Phi_.as_explicit()])
cs_vars, get_lambda_interim_cs = lambdify_cs((u_a, T, Eps.blocks, Sig.blocks), sp_vars, df_dSig_dEps_reduced_)
df_dlambda_2_ = (df_dSig_dEps_cse.T * Phi_cse)[0, 0]
get_f_2_ = sp.lambdify((Eps.blocks, Sig.blocks) + cs_vars + sp_vars + ('**kw',), f_cse, numpy_dirac)
get_df_dlambda_2_ = sp.lambdify((Eps.blocks, Sig.blocks) + cs_vars + sp_vars + ('**kw',), df_dlambda_2_, numpy_dirac)

In [ ]:
import inspect
source_code = inspect.getsource(get_df_dlambda_2_)
print(source_code)


In [ ]:
len(cs_vars), df_dlambda_2_

In [ ]:
_u_a = [1e-6, 1e-6, 0]
_Sig_0_B_00 = get_Sig_(_u_a, 20, _Eps_0_B_00, _Eps_0_B_00, **material_params)
_Sig_0_B_00 = [_Sig_0_i_00[:, 0] for _Sig_0_i_00 in _Sig_0_B_00]
_x_i_cse = get_lambda_interim_cs(_u_a, 20, _Eps_0_B_00, _Sig_0_B_00, **material_params)
_f = get_f_2_(_Eps_0_B_00, _Sig_0_B_00, *_x_i_cse, **material_params)
_df_dlambda = get_df_dlambda_2_(_Eps_0_B_00, _Sig_0_B_00, *_x_i_cse, **material_params)
_f, _df_dlambda

In [ ]:
_x_i_cse_IJ = get_lambda_interim_cs(_u_IJ, 20, _Eps_0_B_IJ, _Sig_0_B_IJ, **material_params)
_f_IJ = get_f_2_(_Eps_0_B_IJ, _Sig_0_B_IJ, *_x_i_cse_IJ, **material_params)
_df_dlambda_IJ = get_df_dlambda_2_(_Eps_0_B_IJ, _Sig_0_B_IJ, *_x_i_cse_IJ, **material_params)
fig, ax = plt.subplots(1, 1, figsize=(7, 3), tight_layout=True)
fig.canvas.header_visible=False
ax.contour(sig_p_range, tau_p_range, _f_IJ, levels=[0]);
ax.contourf(sig_p_range, tau_p_range, _df_dlambda_IJ);
ax.axis('equal');

### Version 3 - conditional evaluation using where

In [ ]:
f_

In [ ]:
f_exprs, f_conds = zip(*f_.args)
f_reduced_, f_cse_list_ = sp.cse((f_exprs + f_conds)[:-1])
cs_vars, get_f_reduced = lambdify_cs((u_a, T, Eps.blocks, Sig.blocks), sp_vars, f_reduced_)
get_f_cse_list = [sp.lambdify((u_a, T, Eps.blocks, Sig.blocks) + cs_vars + sp_vars + ('**kw',), f_cse_, numpy_dirac) 
                  for f_cse_ in f_cse_list_]
n_exprs = len(f_exprs)
get_f_exprs = get_f_cse_list[:n_exprs]
get_f_conds = get_f_cse_list[n_exprs:]

In [ ]:
def get_f_conditional(get_f_conds, get_f_exprs, ext_vars, _Eps, _Sig, **mp):
    """Given a list of conditions and corresponding expressions, evaluate the function
    of external and internal variables and return the expression corresponding to
    the first condition that evaluates to True. If none of the conditions evaluate
    to True, return the expression corresponding to the last condition.

    Parameters
    ----------
    get_f_conds : list
        List of conditions to be evaluated.
    get_f_exprs : list
        List of expressions corresponding to the conditions.
    ext_vars : tuple
        Tuple of external variables.
    _Eps, _Sig : float
        Internal variables.
    **mp : dict
        Dictionary of model parameters.
    
    Returns
    -------
    result : float
        The expression corresponding to the first condition that evaluates to True.
        If none of the conditions evaluate to True, return the expression corresponding
        to the last condition.

        Remark
        ------
        The shape of the result is currently a scalar but can be extended to an nd-array (TODO). 
        This shape can be derived from the symbolic representation of the f_expression. The design question 
        here is if the symbolic source expression should be supplied as an input parameter as well to constitute
        a generic class that takes care of the dimensionality of the result as well.   

    Explanation - hyper shape
    -------------------------
    The function is evaluated in a cascading manner, i.e. the first condition is evaluated
    and if it evaluates to True, the corresponding expression is evaluated and returned.
    If the condition evaluates to False, the next condition is evaluated and so on. The
    last expression is returned if none of the conditions evaluate to True. 

    The structure of int_vars, i.e. _Eps and _Sig, is given as a sequence of arrays with the same shapes.
    Their number and shapes are the same as the block vector symbols Eps and Sig of the 
    constitutive model. Assuming for example Eps[0] is a plastic tensor with the shape (2,2), 
    the int_var_shape. Assuming that the function is evaluated on a n-dimensional array of material
    points with the shape field_shape = (n_x, n_y, n_z), covering a spatial domain by a discretization grid, 
    the shape of the _Eps[0] array is _Eps.shape = (*int_var_shape, *field_shape).

    The structure of ext_vars is a sequence of arrays with potentially a different shape, representing e.g. 
    (strain tensor, temperature, humidity) - observable or controllable variables. Assuming for example that
    the constitutive formulation in the thermodynamic potential uses a strain tensor eps_ab with the shape (2,2),
    the shape of the input variable for a considered field will be (2,2,n_x,n_y,n_z), which must be compatible
    with all other external and internal variables.

    By extracting the field shape from the input variables, it is possible to apply masking to the input variables
    so that cascaded evaluation depending on the satisfied quantification is enabled. 

    Remark - iterative cascaded evaluation
    --------------------------------------
    This feature is relevant 
    both for the evaluation of piecewise decomposed domain, i.e multi-surface / domain threshold and 
    flow potential functions. Even more importantly, the cascaded evaluation is a key feature for the return-mapping 
    and time-stepping algorithm, where only small localizing part of the simulated domain requires multiple iterations 
    to attain an admissible state.

    As these considerations are fully generic, it is worth defining a class that enables this kind of 
    segmented and cascaded evaluation. It is relevant for multidomain inelasticity - that covers the multi-dimensional 
    state representation using piecewise threshold, float potential and dissipation potential functions. 
    The initial version of this functionality is provided here. 

    Another application of the cascaded evaluation is the iterative return mapping algorithm. Then, there is just one 
    condition to be satisfied the whole field_shape. The cascaded evaluation is then used to evaluate the 
    return mapping algorithm for all grid points in the domain. Such cascaded evaluation must be supplied with the 
    predictor and corrector function.

    The class definition consists of 

    F = Free energy potential
    u = external variable
    T = temperature
    Eps = symbolic definition of the internal state (sympy.BlockMatrix)
    Sig = symbolic definition of the internal state (sympy.BlockMatrix)
    mp = list of material parameters
    f = threshold function
    phi_ext = flow potential extension
    """
    _u_a, _T = ext_vars
    _x_i_cse = get_f_reduced(_u_a, _T, _Eps, _Sig, **mp)
    args = (_u_a, _T, _Eps, _Sig, *_x_i_cse)

    # Initialize the result array with the default value
    result = np.full(_u_a.shape[1:], np.nan)
    print(_u_a.shape, result.shape)

    # Initialize the mask to cover all grid elements
    mask = np.ones_like(result, dtype=bool)

    # Efficient cascading condition evaluation and index recovery
    for i, get_f_cond in enumerate(get_f_conds):
        # Apply condition only where the result is undefined and mask is valid
        mask_update = np.logical_and(get_f_cond(*args, **mp), mask)
        # call the function only on the elements where the condition is satisfied 
        _f_i = get_f_exprs[i](*args, **mp)
        # Evaluate conditions only on valid masked elements and update results
        result[mask_update] = _f_i[mask_update]
        # Update the mask by excluding elements where condition is satisfied
        mask[mask_update] = False

    result[mask_update] = get_f_exprs[-1](*args, **mp)[mask_update]
    # result now contains the output after applying the corresponding functions based on conditions
    return result

In [ ]:
_u_a = [1e-6, 1e-6, 0]
_Sig_0_B_00 = get_Sig_(_u_a, 20, _Eps_0_B_00, _Eps_0_B_00, **material_params)
_Sig_0_B_00 = [_Sig_0_i_00[:, 0] for _Sig_0_i_00 in _Sig_0_B_00]
_x_i_cse = get_lambda_interim_cs(_u_a, 20, _Eps_0_B_00, _Sig_0_B_00, **material_params)
_f = get_f_2_(_Eps_0_B_00, _Sig_0_B_00, *_x_i_cse, **material_params)
_df_dlambda = get_df_dlambda_2_(_Eps_0_B_00, _Sig_0_B_00, *_x_i_cse, **material_params)
_f, _df_dlambda

In [ ]:
#get_f_conditional(get_f_conds, get_f_exprs, (np.array(_u_a), 20), _Eps_0_B_00, _Sig_0_B_00, **material_params)

In [ ]:
subs_Sig_Eps_ = dict(zip(Sig.as_explicit(), Sig_.as_explicit()))
f_exprs, f_conds = zip(*f_.args)
df_dEps_exprs = [f_expr.subs(subs_Sig_Eps_).diff(Eps.as_explicit()) for f_expr in f_exprs]
phi_exprs = [(f_expr + phi_ext_) for f_expr in f_exprs]
Phi_exprs = []
for phi_expr in phi_exprs:
    Phi_expr = sp.BlockMatrix([[-sign_i_ * phi_expr.diff(Sig_i_)] for sign_i_, Sig_i_ in zip(Sig_signs, Sig.blocks)]).as_explicit()
    Phi_exprs.append(Phi_expr)

In [ ]:
n_exprs = len(f_exprs)
all_expr = (f_exprs + tuple(df_dEps_exprs) + tuple(Phi_exprs) + f_conds)
f_reduced_, f_cse_list_ = sp.cse(all_expr)
f_cse_ = f_cse_list_[:n_exprs]
df_dEps_cse_ = f_cse_list_[n_exprs:n_exprs*2]
Phi_cse_ = f_cse_list_[n_exprs*2:n_exprs*3]
f_conds_cse_ = f_cse_list_[n_exprs*3:-1]
df_dlambda_cse_ = [(df_dEps_cse_[i].T * Phi_cse_[i])[0,0] for i in range(n_exprs)]
args_ = f_cse_ + df_dlambda_cse_ + f_conds_cse_

In [ ]:
cs_vars, get_f_reduced = lambdify_cs((u_a, T, Eps.blocks, Sig.blocks), sp_vars, f_reduced_)
get_f_cse_list = [sp.lambdify((u_a, T, Eps.blocks, Sig.blocks) + cs_vars + sp_vars + ('**kw',), 
                              f_cse_, numpy_dirac) 
                  for f_cse_ in args_]
get_f_exprs = get_f_cse_list[:n_exprs]
get_df_dlambda_exprs = get_f_cse_list[n_exprs:n_exprs*2]
get_f_conds_exprs = get_f_cse_list[n_exprs*2:]

In [ ]:
_f_4_IJ = get_f_conditional(get_f_conds_exprs, get_f_exprs,  (_u_IJ, 20), _Eps_0_B_IJ, _Sig_0_B_IJ, **material_params)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 3), tight_layout=True)
fig.canvas.header_visible=False
ax.contour(sig_p_range, tau_p_range, _f_4_IJ, levels=[0]);
#ax.contourf(sig_p_range, tau_p_range, _df_dlambda_IJ);
ax.axis('equal');